In [ ]:
import torch
import cv2
import numpy as np
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
from PIL import Image
import plotly.graph_objects as go
from transformers import pipeline
import os

def load_model():
    device = "cuda" if torch.cuda.is_available() else "cpu"
    checkpoint = "depth-anything/Depth-Anything-V2-base-hf"
    pipe = pipeline("depth-estimation", model=checkpoint, device=device)
    return pipe

def estimate_depth(model, img):
    if isinstance(img, np.ndarray):
        img_pil = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    else:
        img_pil = img

    predictions = model(img_pil)
    depth_map = predictions["depth"]
    depth_map_np = np.array(depth_map).squeeze()
    depth_map_resized = cv2.resize(depth_map_np, (img_pil.size[0], img_pil.size[1]))
    
    return depth_map_resized

def process_image(model, image_path):
    image = cv2.imread(image_path)
    if image is None:
        return None

    depth_map = estimate_depth(model, image)
    return image, depth_map

def combine_depth_maps(depth_maps):
    # Simple averaging of depth maps
    return np.mean(depth_maps, axis=0)

def create_point_cloud(images, depth_maps, sample_rate=10):
    points = []
    colors = []
    
    for img, depth in zip(images, depth_maps):
        h, w = depth.shape
        for y in range(0, h, sample_rate):
            for x in range(0, w, sample_rate):
                z = depth[y, x]
                if z > 0:
                    points.append([x, y, z])
                    colors.append(img[y, x])
    
    return np.array(points), np.array(colors)

def plot_3d_point_cloud(points, colors):
    fig = go.Figure(data=[go.Scatter3d(
        x=points[:, 0],
        y=points[:, 1],
        z=points[:, 2],
        mode='markers',
        marker=dict(
            size=2,
            color=['rgb({},{},{})'.format(r, g, b) for r, g, b in colors],
            opacity=0.8
        )
    )])

    fig.update_layout(
        scene=dict(
            xaxis_title='X',
            yaxis_title='Y',
            zaxis_title='Z',
            aspectmode='data'
        ),
        margin=dict(l=0, r=0, b=0, t=0)
    )

    return fig

def process_multiple_images(image_paths):
    model = load_model()
    images = []
    depth_maps = []

    for path in image_paths:
        result = process_image(model, path)
        if result is not None:
            image, depth_map = result
            images.append(image)
            depth_maps.append(depth_map)

    combined_depth = combine_depth_maps(depth_maps)
    points, colors = create_point_cloud(images, depth_maps)
    fig = plot_3d_point_cloud(points, colors)
    
    return fig

# Example usage
image_paths = ['path/to/image1.jpg', 'path/to/image2.jpg', 'path/to/image3.jpg']
fig = process_multiple_images(image_paths)
fig.show()